In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from tqdm import tqdm
recommender = pd.read_csv('places.csv')
df = recommender.reset_index(drop=True)


In [5]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(df['Place'])

In [6]:
# Compute cosine similarity matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
results = {}
for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['PlaceID'].iloc[i]) for i in similar_indices]
    results[idx] = similar_items[1:]

In [8]:
def item(id):
    return df.loc[df['PlaceID'] == id]['Place'].tolist()[0].split(' - ')[0]

In [9]:
def get_idx(id):
    return df[df['PlaceID'] == id].index.tolist()[0]

In [10]:
def recommend(item_id, num):
    output = {"Suggested": []}
    recs = results[get_idx(item_id)][:num]    # Get recommended places
    for rec in recs:
        place_name = item(rec[1])
        image_url = df.loc[df["Place"] == place_name]["ImageUrl"].tolist()
        output["Suggested"].append([place_name, image_url])
    return output